<div style="background: #cceeaa; padding: 5px; border: 1px solid green; border-radius: 5px;">
    <font color='green'> <b><u>КОММЕНТАРИЙ РЕВЬЮЕРА</u></b>

Дмитрий, доброго дня! рад приветствовать тебя вновь)<br />
(а дальше шаболонная обязательная часть, как ты заметил по другим спринтам)
<br />
Меня зовут Николай Шавлюго. <br />И на этом этапе твоего движения к новой профессии от меня будут комментарии по написанному коду. <br />Чтобы меньше "мусорить эфир" и чтобы максимально наглядно отделяться от основного кода, есть предложение условиться в некоторых обозначениях:<br /> <br />
<u><b>ТАКОЙ ШРИФТ</b></u> - всегда начало комментария <br />
<font color='green'>такой шрифт</font> - комментарии о том, что всё ОК <br/>
<font color='orange'>такой шрифт</font> - комментарии о том, что всё ОК по результату,<br> однако есть на что обратить внимание в плане применения техник, или есть способы сделать более короткий или быстрый код <br/>
<font color='red'>такой шрифт</font> - комментарии о том, что есть критичный момент, влияющий на бизнес-результат проекта.<br/>
</font><br /><br />
<font color='green'>Моей целью является не "уличить" в не знании, а просто высказать сверху твоих знаний - свой опыт, что бы ты мог использовать его для своего дальнейшего успеха) И очень здорово будет, если тебе удастся задавать вопросы, да и вообще - всячески доставать меня, если я по каким-то причинам не приму проект:) При этом, из своего опыта скажу, хорошо и важно, когда переписка ревьюера и студента - сохраняется на следующие проекты и даже на будущую практическую деятельность.<br>
<BR> В ПУТЬ!<br>

## Проект для «Викишоп»
____

### Описание проекта

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.
Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.
 * Постройте модель со значением метрики качества F1 не меньше 0.75.
____

**Признаки:**  
* `text` — текст комментария;
* `toxic` — целевой признак.
____

### Структура проекта
<a href='#section1'></a>
1. [Подготовка данных](#section1)  
    1.1 [Первичный осмотр](#section1.1)  
    1.2. [Очистка и лемматизация комментариев](#section1.2)   
<a href='#section2'></a>  
2. [Построение модели](#section2)  
    2.1. [Подготовка данных к обучению](#section2.1)  
    2.2. [LogisticRegression](#section2.2)  
    2.3. [DecisionTreeClassifier](#section2.3)    
    2.4. [LGBMClassifier](#section2.4)  
    2.5. [LinearSVC](#section2.5)  
    2.6. [Выводы обучения](#section2.6)  
    2.7. [Тестирование](#section2.7)  
<a href='#section3'></a>  
3. [Выводы исследования](#section3)
____
____

<a id='section1'></a>
## 1. Подготовка данных

In [1]:
import pandas as pd
import numpy as np
# String
import re
import string
# nltk
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords as nltk_stopwords
from nltk import word_tokenize, pos_tag
from nltk.tokenize import sent_tokenize, word_tokenize
# Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# Модели
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import LinearSVC
# Инструменты
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.utils import shuffle
import warnings
from time import time
# Метрики
from sklearn.metrics import precision_score, recall_score, accuracy_score,  f1_score,  roc_auc_score, roc_curve, make_scorer
# Графика
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')
%matplotlib inline
sns.set_style('darkgrid')
sns.set_context("paper")
# nltk
stop_words = set(nltk_stopwords.words('english'))
punctuation = string.punctuation 
wordnet_lemmatizer = WordNetLemmatizer()

<a id='section1.1'></a>
### 1.1 Первичный осмотр

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Посмотрим на классы

In [4]:
print(data['toxic'].value_counts())

class_ratio = round(data['toxic'].value_counts()[0] / data['toxic'].value_counts()[1], 2)
class_ratio

0    143346
1     16225
Name: toxic, dtype: int64


8.83

Имеем сильный дисбаланс классов: `1 : 8.83`.  
Воспользуемся балансировкой весов обучения.

Лемматизируем текст перед этим и разобьём на выборки.

<div style="background: #cceeaa; padding: 5px; border: 1px solid green; border-radius: 5px;">
    <font color='green'> <b><u>КОММЕНТАРИЙ РЕВЬЮЕРА</u></b>
</font>
<font color='green'><br>
Ок. данные на базе)

<a id='section1.2'></a>
### 1.2 Очистка и лемматизация комментариев

Изначально пользовались `Mystem` из библиотеки `pymystem3`, однако процесс очистки и лемматизации улетел за полчаса => решено было использовать `WordNetLemmatizer` и функции `map`, `filter`.

In [5]:
def preparation_of_text(text):
    tokens = [word for w in sent_tokenize(text) for word in word_tokenize(w)]
    # Remove punctuation inside words
    tokens = list(filter(lambda token: token not in punctuation, tokens)) 
    # Remove stopwords
    tokens = list(filter(lambda token: token.lower() not in stop_words, tokens))
    
    filtered_tokens = []
    for token in tokens:
        if re.search("[a-zA-Z]", token):
            filtered_tokens.append(token)
    # lemmatize text
    filtered_tokens = list(map(lambda token: wordnet_lemmatizer.lemmatize(token.lower()), filtered_tokens)) 
    filtered_tokens = list(filter(lambda token: token not in punctuation, filtered_tokens))
    return " ".join(filtered_tokens)

In [6]:
%%time

data["clear_text"] = data["text"].map(preparation_of_text)

CPU times: user 4min 44s, sys: 780 ms, total: 4min 45s
Wall time: 4min 50s


In [7]:
data = data.drop("text", axis=1)
data.head()

,toxic,clear_text
0,0,explanation edits made username hardcore metal...
1,0,d'aww match background colour 'm seemingly stu...
2,0,hey man 'm really trying edit war 's guy const...
3,0,ca n't make real suggestion improvement wonder...
4,0,sir hero chance remember page 's


In [8]:
# def train_valid_test_split(data : pd.DataFrame, target_column : str, proportions=[0.6, 0.2, 0.2]):
#     """
#     Function splits by train, validation, test parts income dataframe.
#     In:
#         df : pandas.DataFrame() - income dataframe.
#         proportions : list() - list of proportions values to split method.
#     """
#     if sum([*proportions]) != 1:
#         raise AttributeError('Wrong "proportions" values. Sum of "proportions" values > 1.0. Must be equal to 1.0.')
#     data_list = [train, valid, test] = np.split(
#         data.sample(frac=1, random_state=314), 
#         [int(proportions[0] * len(data)), int(sum(proportions[:2]) * len(data))]
#     )
#     features, targets = list(), list()
#     for data in data_list:
#         features.append(data.drop(target_column, axis=1))
#         targets.append(data[target_column])
#     return [*features, *targets]

In [9]:
target = data["toxic"]
features = data.drop("toxic", axis=1)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=.2, random_state=314)
print("X_shapes", X_train.shape, X_test.shape, sep=' || ')

X_shapes || (127656, 1) || (31915, 1)


In [11]:
X_train.head()

,clear_text
71831,think mucha lucha homosexual show 've ever wit...
38139,swore well
6079,corrected thee countless ahs china
31624,n't offended 'm sorry keep fixing indentation ...
129677,previous full lupine state allowed lift ton sa...


Данные на базе😉

<div style="background: #cceeaa; padding: 5px; border: 1px solid green; border-radius: 5px;">
    <font color='green'> <b><u>КОММЕНТАРИЙ РЕВЬЮЕРА</u></b>
</font>
<font color='green'><br>
Точно:)
    
При этом - важная часть работы сделана - чистка и лемматизация! <br>
Это как шашлык готовить: сначала важный этап маринада, а потом приготовление:)

<a id='section2'></a>
## 2. Построение модели

<a id='section2.1'></a>
### 2.1 Подготовка данных к обучению

In [12]:
train_corpus = X_train["clear_text"].values.astype('U')
test_corpus = X_test["clear_text"].values.astype('U')
# Vectorizer
count_tf_idf = TfidfVectorizer(stop_words=stop_words)
# Fit_trasform train & transform others.
X_train = count_tf_idf.fit_transform(train_corpus)
X_test = count_tf_idf.transform(test_corpus)

<div style="background: #cceeaa; padding: 5px; border: 1px solid green; border-radius: 5px;">
    <font color='green'> <b><u>КОММЕНТАРИЙ РЕВЬЮЕРА</u></b>
</font>
<font color='green'><br>
ок, в такой последовательности)

In [13]:
results = []

In [14]:
scorer = make_scorer(f1_score)

In [15]:
def cross_validation(model, X, y, param_grid, scoring):
    model_cv = GridSearchCV(
        model(),
        param_grid = param_grid,
        cv = 4,
        scoring = scoring,
        n_jobs = -1,
        verbose = 10,
        return_train_score = True
    )
    # Подбор гиперпараметров
    model_cv = model_cv.fit(X, y)
    # Best etimator
    model_best = model_cv.best_estimator_
    cv_results = model_cv.cv_results_

    # Train
    start_time = time()
    model_best.fit(X, y)
    train_time = time() - start_time
    # Predict
    start_time = time()
    predictions = model_best.predict(X)
    predict_time = time() - start_time
    # F1-score metrics
    f1_score_metrics = f1_score(y, predictions)
    # Result list
    rlt = [{
        "Model name" : type(model_best).__name__,
        "Train time" : round(train_time, 4),
        "Predict time" : round(predict_time, 4),
        "F1 train_score" : model_cv.best_score_.round(4),
        "F1 predict_score" : round(f1_score_metrics, 4),
        "Parameters" : model_cv.best_params_
    }]
    return model_best, model_cv.best_params_, rlt

<a id='section2.2'></a>
### 2.2 LogisticRegression

In [16]:
logreg_params_dict = {
    "random_state" : [314],
    "penalty" : ["l1"],
    "class_weight" : [None, "balanced"],
    "max_iter" : range(5, 20, 2)
}

In [17]:
%%time

logreg, logreg_params, logreg_result = cross_validation(
    model = LogisticRegression,
    X = X_train,
    y = y_train,
    param_grid = logreg_params_dict,
    scoring = scorer
)

results += logreg_result
results[-1]

Fitting 4 folds for each of 16 candidates, totalling 64 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   16.0s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   18.2s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   25.2s
[Parallel(n_jobs=-1)]: Done  56 out of  64 | elapsed:   33.0s remaining:    4.6s
[Parallel(n_jobs=-1)]: Done  64 out of  64 | elapsed:   37.3s finished


Wall time: 38.6 s


{'Model name': 'LogisticRegression',
 'Train time': 0.392,
 'Predict time': 0.011,
 'F1 train_score': 0.7682,
 'F1 predict_score': 0.7955,
 'Parameters': {'class_weight': None,
  'max_iter': 5,
  'penalty': 'l1',
  'random_state': 314}}

<a id='section2.3'></a>
### 2.3 DecisionTreeClassifier

In [18]:
tree_params_dict = {
    "random_state" : [314],
    "max_depth" : range(2, 4),
    "min_samples_split" : range(2, 4),
    "min_samples_leaf" : range(2, 4),
    "class_weight" : [None, "balanced"]
}

In [19]:
%%time

tree, tree_params, tree_result = cross_validation(
    model = DecisionTreeClassifier,
    X = X_train,
    y = y_train,
    param_grid = tree_params_dict,
    scoring = scorer
)

results += tree_result
results[-1]

Fitting 4 folds for each of 16 candidates, totalling 64 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   14.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   29.7s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   30.3s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  56 out of  64 | elapsed:  1.8min remaining:   15.6s
[Parallel(n_jobs=-1)]: Done  64 out of  64 | elapsed:  2.1min finished


Wall time: 2min 8s


{'Model name': 'DecisionTreeClassifier',
 'Train time': 1.0642,
 'Predict time': 0.1127,
 'F1 train_score': 0.4265,
 'F1 predict_score': 0.4288,
 'Parameters': {'class_weight': None,
  'max_depth': 3,
  'min_samples_leaf': 2,
  'min_samples_split': 2,
  'random_state': 314}}

<a id='section2.4'></a>
### 2.4 LGBMClassifier

In [20]:
lgb_params_dict = {
    "random_state" : [314],
    "learning_rate" : [0.25, 0.3],
    "n_estimators" : [150, 200, 250],
    "max_depth" : range(3, 5),
    "class_weight" : ["balanced"],
}

In [21]:
%%time

lgb, lgb_params, lgb_result = cross_validation(
    model = LGBMClassifier,
    X = X_train,
    y = y_train,
    param_grid = lgb_params_dict,
    scoring = scorer
)

results += lgb_result
results[-1]

Fitting 4 folds for each of 12 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   53.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done  38 out of  48 | elapsed:  4.6min remaining:  1.2min
[Parallel(n_jobs=-1)]: Done  43 out of  48 | elapsed:  5.0min remaining:   34.5s
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  5.4min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  5.4min finished


Wall time: 5min 55s


{'Model name': 'LGBMClassifier',
 'Train time': 15.9514,
 'Predict time': 0.5525,
 'F1 train_score': 0.7427,
 'F1 predict_score': 0.783,
 'Parameters': {'class_weight': 'balanced',
  'learning_rate': 0.3,
  'max_depth': 4,
  'n_estimators': 200,
  'random_state': 314}}

<a id='section2.5'></a>
### 2.5 LinearSVC

In [40]:
svc_params_dict = {
    "C" : [x / 10 for x in range(1, 21)],
    "max_iter" : [5, 7, 10, 25, 100],
    "class_weight" : ["balanced"],
    "random_state" : [314]
}

In [41]:
%%time

svc, svc_params_dict, svc_result = cross_validation(
    model = LinearSVC,
    X = X_train,
    y = y_train,
    param_grid = svc_params_dict,
    scoring = scorer
)

results += svc_result
results[-1]

Fitting 4 folds for each of 100 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   12.3s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:   15.2s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:   18.2s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:   21.5s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:   23.6s
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:   27.8s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   31.9s
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed:   36.1s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   

Wall time: 1min 25s


{'Model name': 'LinearSVC',
 'Train time': 0.2563,
 'Predict time': 0.013,
 'F1 train_score': 0.7746,
 'F1 predict_score': 0.8819,
 'Parameters': {'C': 0.7,
  'class_weight': 'balanced',
  'max_iter': 5,
  'random_state': 314}}

In [42]:
def view_results(results : list):    
    return (pd.DataFrame(results)
        .style
        .highlight_min(color='orange')
        .highlight_max(color='lightgreen')
    )

In [43]:
view_results(results)

,Model name,Train time,Predict time,F1 train_score,F1 predict_score,Parameters
0,LogisticRegression,0.392,0.011,0.7682,0.7955,"{'class_weight': None, 'max_iter': 5, 'penalty': 'l1', 'random_state': 314}"
1,DecisionTreeClassifier,1.0642,0.1127,0.4265,0.4288,"{'class_weight': None, 'max_depth': 3, 'min_samples_leaf': 2, 'min_samples_split': 2, 'random_state': 314}"
2,LGBMClassifier,15.9514,0.5525,0.7427,0.783,"{'class_weight': 'balanced', 'learning_rate': 0.3, 'max_depth': 4, 'n_estimators': 200, 'random_state': 314}"
3,LinearSVC,0.2563,0.013,0.7746,0.8819,"{'C': 0.7, 'class_weight': 'balanced', 'max_iter': 5, 'random_state': 314}"


<a id='section2.6'></a>
### 2.6 Выводы обучения

Согласно результатам обучения модель `LinearSVC` имеет наилучшие значения как по метрике `F1-score` так и по времени обучения.
Дальше следует `LogisticRegression` и `LGBMClassifier`. Эти модели идут на тест.

<div style="background: #cceeaa; padding: 5px; border: 1px solid green; border-radius: 5px;">
    <font color='green'> <b><u>КОММЕНТАРИЙ РЕВЬЮЕРА</u></b>
</font>
<font color='green'><br>
Дмитрий, задачка такая, что часто и железо не тянет, больше на терпение:)<br>
Интересные исследования по применению разных моделей ты провёл!<br>
Однозначно видно, что ты понимаешь что делать.<br>
И если потребуется на практике - реально сможешь применить. А это тот результат, что и нужен на обучении.<br>
Если же говорить о потенциале по точности модели, то тут он, в том числе, кроется в исследовании самих фраз: длинные/короткие (возможно даже как фактор внести размер фразы - ну или поисследовать этот момент), установить минимальное количество букв во фразе... <br>
      
Всё чётко у тебя и толково!<br>    
Спасибо) УСПЕШНОГО И ДАЛЬШЕ ОБУЧЕНИЯ!

<a id='section2.7'></a>
### 2.7 Тестирование

In [44]:
def get_test(model, X_test, y_test):
    # Predict
    start = time()
    predictions = model.predict(X_test)
    predict_time = time() - start
    # F1-score
    f1_score_metrics = f1_score(y_test, predictions)
    
    return [{
            "Model name" : type(model).__name__,
            "Predict test time" : round(predict_time, 4),
            "F1 test_score" : round(f1_score_metrics, 4),
            "Parameters" : model.get_params(model),
        }]

In [45]:
test_results = []

In [46]:
test_results += get_test(logreg, X_test, y_test)
test_results += get_test(lgb, X_test, y_test)
test_results += get_test(svc, X_test, y_test)

In [47]:
def color_completed(value):
    color = 'orange' if value==True else 'black'
    return 'color: %s'%color

In [48]:
for result in test_results:
    f1_threshold = 0.75
    if result["F1 test_score"] >= f1_threshold:
        result["Completed?"] = True
    else:
        result["Completed?"] = False

In [49]:
view_results(test_results).applymap(color_completed)

,Model name,Predict test time,F1 test_score,Parameters,Completed?
0,LogisticRegression,0.005,0.7746,"{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 5, 'multi_class': 'warn', 'n_jobs': None, 'penalty': 'l1', 'random_state': 314, 'solver': 'warn', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}",True
1,LGBMClassifier,0.1426,0.7461,"{'boosting_type': 'gbdt', 'class_weight': 'balanced', 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.3, 'max_depth': 4, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 200, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'random_state': 314, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'silent': True, 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0}",False
2,LinearSVC,0.004,0.7548,"{'C': 0.7, 'class_weight': 'balanced', 'dual': True, 'fit_intercept': True, 'intercept_scaling': 1, 'loss': 'squared_hinge', 'max_iter': 5, 'multi_class': 'ovr', 'penalty': 'l2', 'random_state': 314, 'tol': 0.0001, 'verbose': 0}",True


<a id='section3'></a>
## 3. Выводы исследования

В результате построена модель оценки токсичности комментария.
Наилучшие результаты показывают модели `LogisticRegression` и `LinearSVC` со значениями метрик больше порогового $F1 = 0.75$.
Учитывая их преимущества в скорости обучения перед градиентным бустингом, на выбор заказчику рекомендуется любая из них.
Но из-за большего значения метрики `F1-score` большее предпочтение отдаём логистической регрессии.